In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
oPath = 'database/tables'
# Read all the sheets from the excel file.
students = pd.read_csv('{}/students.csv'.format(oPath), header = 0)
classes = pd.read_csv('{}/classes.csv'.format(oPath), header = 0)
tests = pd.read_csv('{}/tests.csv'.format(oPath), header = 0)
test_results = pd.read_csv('{}/test_results.csv'.format(oPath), header = 0)
attendance = pd.read_csv('{}/attendance.csv'.format(oPath), header = 0)
hw = pd.read_csv('{}/hw_assignments.csv'.format(oPath), header = 0)
hw_results = pd.read_csv('{}/hw_grades.csv'.format(oPath), header = 0)
participation = pd.read_csv('{}/participation.csv'.format(oPath), header = 0)

In [ ]:
# Define a function to show values
def show_values_on_bars(axs):
    def _show_on_single_plot(ax):        
        for p in ax.patches:
            _x = p.get_x() + p.get_width() / 2
            _y = p.get_y() + p.get_height() + 0.5
            value = '{:.0f}'.format(p.get_height())
            ax.text(_x, _y, value, ha="center") 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

# Participation Reports by Class

In [ ]:
dfVOL = pd.merge(students, participation, on = 'STUDENT_ID')
value_counts = dfVOL['STUDENT_ID'].value_counts()
# Convert value_counts to a DataFrame
dfPart = pd.DataFrame(value_counts)
# Reset its index
dfPart = dfPart.reset_index()
# Rename the columns to make them more readable
dfPart.columns = ['STUDENT_ID', 'PARTICIPATIONS COUNT']
# Merge the dfPart and students DataFrames
dfPart = pd.merge(dfPart, students, how = 'outer', on = 'STUDENT_ID')

In [ ]:
# Get the mean number of participations for each class
class_mean = dfPart.groupby('CLASS_ID')['PARTICIPATIONS COUNT'].mean()
# Convert class_mean to a DataFrame
class_mean = pd.DataFrame(class_mean)
# Reset its index
class_mean = class_mean.reset_index()
# Rename the columns
class_mean.columns = ['CLASS_ID', 'MEAN # OF PARTICIPATIONS']
# Merge the class_mean and the dfPart DataFrames 
dfNoPall = pd.merge(dfPart, class_mean, how = 'outer', on = 'CLASS_ID')

## Participations Count by Class

### Prepare DataFrames

In [ ]:
# Get a list of all CLASS_IDs in ascending order
dfSorted = dfNoPall.sort_values(by = 'CLASS_ID')
lClasses = dfSorted['CLASS_ID'].unique()
# Loop through all CLASS_IDs to create a DataFrame for each class
dfNoP = {}
for i in lClasses:
    dfNoP[i] = dfNoPall.where(dfNoPall['CLASS_ID'] == i)
    dfNoP[i].drop(columns = ['STUDENT_FIRST_DAY', 'STUDENT_LAST_DAY'], axis = 0, inplace = True)
    dfNoP[i] = dfNoP[i].dropna(how = 'all')
    dfNoP[i].reset_index(inplace = True, drop = True)
    dfNoP[i] = dfNoP[i].sort_values(by = 'PARTICIPATIONS COUNT', ascending = False).reset_index(drop = True)
    dfNoP[i] = dfNoP[i].astype({'STUDENT_ID': 'int32'})

### Reports

In [ ]:
for i in lClasses:
    tTitle = "Particpation Count by Student for Class {}".format(i)
    # Plot participations count
    pBlah = dfNoP[i].plot.bar(x = "STUDENT_NAME_ENGLISH", y = 'PARTICIPATIONS COUNT', color = 'darkgreen', figsize=(20,12), legend = False)
    # Plot the average as a line on the second axis
    dfNoP[i].plot(x = "STUDENT_NAME_ENGLISH", y = 'MEAN # OF PARTICIPATIONS', ax = pBlah, color = 'lightblue', figsize=(20,12), lw = 4, legend = True)
    # Format the graph
    pBlah.set_title(tTitle, size = 30)
    pBlah.set_xlabel("Name", size = 18)
    pBlah.set_ylabel("Participation Count", size = 18)
    pBlah.legend(["Mean: " + str(round(dfNoP[i]['MEAN # OF PARTICIPATIONS'][0]))])
    # Label the bars
    show_values_on_bars(pBlah)
    # Save the figure
    pBlah.figure.savefig("images\{}.png".format(tTitle))

## Participation Average Per Day

### Prepare DataFrames

In [263]:
dfByDate = dfVOL.groupby(['STUDENT_ID', 'PARTICIPATION_DATETIME'])['PARTICIPATION_DATETIME'].count()
dfByDate = pd.DataFrame(dfByDate)
dfByDate = dfByDate.rename(columns = {'STUDENT_ID': 'STUDENT_ID', 'PARTICIPATION_DATETIME': 'DATETIME', 'PARTICIPATION_DATETIME': 'COUNT'})
dfByDate

COUNT
STUDENT_ID PARTICIPATION_DATETIME       
0          1/10/2015                   6
           1/11/2014                   4
           1/17/2015                   2
           1/18/2014                   5
           1/24/2015                   7
...                                  ...
499        9/18/2013                   4
           9/24/2014                   1
           9/25/2013                   4
           9/3/2014                    2
           9/4/2013                    9

[62527 rows x 1 columns]

In [272]:
dfa = dfByDate
dfa['MEAN'] = dfa.groupby(['STUDENT_ID', 'PARTICIPATION_DATETIME'])['COUNT'].mean()
dfa

COUNT  MEAN
STUDENT_ID PARTICIPATION_DATETIME             
0          1/10/2015                   6     6
           1/11/2014                   4     4
           1/17/2015                   2     2
           1/18/2014                   5     5
           1/24/2015                   7     7
...                                  ...   ...
499        9/18/2013                   4     4
           9/24/2014                   1     1
           9/25/2013                   4     4
           9/3/2014                    2     2
           9/4/2013                    9     9

[62527 rows x 2 columns]

In [271]:
student_id = 499
dfa.loc[student_id]

,COUNT,MEAN
PARTICIPATION_DATETIME,,
1/1/2014,1,1
1/14/2015,3,3
1/15/2014,3,3
1/21/2015,3,3
1/22/2014,4,4
...,...,...
9/18/2013,4,4
9/24/2014,1,1
9/25/2013,4,4


### Reports